In [619]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Load Data

In [620]:
# File with data must be in the same directory as notebook
raw_data = pd.read_csv("apple_quality.csv")

# Remove last row with signature
raw_data.drop(raw_data.index[-1], inplace=True)
raw_data.drop(['A_id'], axis=1, inplace=True)

# Display data sample
raw_data.head()

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483,good
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367,good
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473,bad
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723217,good
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984036,good


Show data info

In [621]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Size         4000 non-null   float64
 1   Weight       4000 non-null   float64
 2   Sweetness    4000 non-null   float64
 3   Crunchiness  4000 non-null   float64
 4   Juiciness    4000 non-null   float64
 5   Ripeness     4000 non-null   float64
 6   Acidity      4000 non-null   object 
 7   Quality      4000 non-null   object 
dtypes: float64(6), object(2)
memory usage: 250.1+ KB


Display datatypes

In [622]:
raw_data.dtypes

Size           float64
Weight         float64
Sweetness      float64
Crunchiness    float64
Juiciness      float64
Ripeness       float64
Acidity         object
Quality         object
dtype: object

Display number of unique values

In [623]:
raw_data.nunique()

Size           4000
Weight         4000
Sweetness      4000
Crunchiness    4000
Juiciness      4000
Ripeness       4000
Acidity        4000
Quality           2
dtype: int64

Use encoder to convert data into numeric values

In [624]:
from sklearn.preprocessing import LabelEncoder

# Use encoder to convert text data into numbers
label_encoder = LabelEncoder()
raw_data['Quality'] = label_encoder.fit_transform(raw_data['Quality'])

# Display transformed data head
raw_data.head()

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483,1
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367,1
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473,0
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723217,1
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984036,1


Find number of columns and rows

In [625]:
raw_data.shape

(4000, 8)

# Create PyTorch neural network

In [626]:
# Create network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(7, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return F.sigmoid(self.layer3(x))
    
# Select correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create an instance of the model
model = NeuralNetwork().to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()    # This criterion support only sigmoid output
optimizer = optim.AdamW(model.parameters(), lr=1e-4)


# Prepare Data for learning

Select X and Y (data snd labels)

In [627]:
# Extract the first 7 columns as X
X = raw_data.iloc[:, :7].values.astype('float32')

# Extract the 8th column as Y
Y = raw_data.iloc[:, 7].values.astype('float32')



Data preprocessing

In [628]:
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler
scaler = StandardScaler()

# Feed data into scaler
X = scaler.fit_transform(X)

Convert to tensor

In [629]:
# Convert it into tensors
X = torch.tensor(X, dtype=torch.float32, device=device)
Y = torch.tensor(Y, dtype=torch.float32, device=device)

# Split data into training and testing sets

In [630]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_training, X_testing, y_training, y_testing = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# Train the model

In [631]:
# Train the model
num_epochs = 10000
for epoch in range(num_epochs):

    # Forward pass
    outputs = model(X_training)
    loss = criterion(outputs, y_training.view(-1, 1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [5/10000], Loss: 0.6883
Epoch [10/10000], Loss: 0.6818
Epoch [15/10000], Loss: 0.6755
Epoch [20/10000], Loss: 0.6693
Epoch [25/10000], Loss: 0.6631
Epoch [30/10000], Loss: 0.6569
Epoch [35/10000], Loss: 0.6506
Epoch [40/10000], Loss: 0.6443
Epoch [45/10000], Loss: 0.6379
Epoch [50/10000], Loss: 0.6315
Epoch [55/10000], Loss: 0.6249
Epoch [60/10000], Loss: 0.6183
Epoch [65/10000], Loss: 0.6115


Epoch [70/10000], Loss: 0.6047
Epoch [75/10000], Loss: 0.5978
Epoch [80/10000], Loss: 0.5909
Epoch [85/10000], Loss: 0.5840
Epoch [90/10000], Loss: 0.5770
Epoch [95/10000], Loss: 0.5700
Epoch [100/10000], Loss: 0.5630
Epoch [105/10000], Loss: 0.5561
Epoch [110/10000], Loss: 0.5493
Epoch [115/10000], Loss: 0.5424
Epoch [120/10000], Loss: 0.5357
Epoch [125/10000], Loss: 0.5291
Epoch [130/10000], Loss: 0.5225
Epoch [135/10000], Loss: 0.5161
Epoch [140/10000], Loss: 0.5098
Epoch [145/10000], Loss: 0.5036
Epoch [150/10000], Loss: 0.4976
Epoch [155/10000], Loss: 0.4917
Epoch [160/10000], Loss: 0.4860
Epoch [165/10000], Loss: 0.4804
Epoch [170/10000], Loss: 0.4751
Epoch [175/10000], Loss: 0.4699
Epoch [180/10000], Loss: 0.4649
Epoch [185/10000], Loss: 0.4600
Epoch [190/10000], Loss: 0.4553
Epoch [195/10000], Loss: 0.4507
Epoch [200/10000], Loss: 0.4463
Epoch [205/10000], Loss: 0.4420
Epoch [210/10000], Loss: 0.4378
Epoch [215/10000], Loss: 0.4338
Epoch [220/10000], Loss: 0.4299
Epoch [225/100

# Evaluate the model

In [632]:
# source data without backpropagation
with torch.no_grad():
    outputs = model(X_testing)
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_testing.view(-1, 1)).float().mean()
    print(f'Accuracy on test data: {accuracy.item():.4f}')


Accuracy on test data: 0.9437
